In [29]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import json
import numpy as np
import pandas as pd

In [30]:
#Reading the link and getting right tag from it

url = input("")
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')
tags = soup('script')
print(tags)

https://www.fotmob.com/match/3714066/matchfacts/villarreal-vs-manchester-united
[<script async="" src="https://www.googletagmanager.com/gtag/js"></script>, <script>
      window.dataLayer = window.dataLayer || [];
      function gtag(){dataLayer.push(arguments);}
      gtag('js', new Date());
      gtag('config', 'UA-59258722-1');
    </script>, <script async="" src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>, <script>
      (function(h,o,t,j,a,r){
          h.hj=h.hj||function(){(h.hj.q=h.hj.q||[]).push(arguments)};
          h._hjSettings={hjid:2585474,hjsv:6};
          a=o.getElementsByTagName('head')[0];
          r=o.createElement('script');r.async=1;
          r.src=t+h._hjSettings.hjid+j+h._hjSettings.hjsv;
          a.appendChild(r);
      })(window,document,'https://static.hotjar.com/c/hotjar-','.js?sv=');
    </script>, <script>
        // WARNING: See the following for security issues around embedding JSON in HTML:
        // https://redux.js.

In [31]:
#Converting data to strings to slice, and then converting back to JSON

strdata = str(tags[4])
ind_start = strdata.index("{")
ind_end = strdata.index("leagueSeasonStats")
strdata = strdata[ind_start:ind_end+22]
data = json.loads(strdata)
data

{'lsLeagues': {'20211126': {'data': [{'isGroup': True,
     'groupName': 'A',
     'ccode': 'INT',
     'id': 875027,
     'primaryId': 73,
     'name': 'Europa League Grp. A',
     'matches': [{'id': 3714170,
       'leagueId': 875027,
       'time': '25.11.2021 21:00',
       'home': {'id': 8595,
        'score': 1,
        'name': 'Brøndby IF',
        'longName': 'Brøndby IF'},
       'away': {'id': 9748, 'score': 3, 'name': 'Lyon', 'longName': 'Lyon'},
       'statusId': 6,
       'tournamentStage': '5',
       'status': {'finished': True,
        'started': True,
        'cancelled': False,
        'scoreStr': '1 - 3',
        'startDateStr': 'Nov 26, 2021',
        'startDateStrShort': '26. Nov.',
        'reason': {'short': 'FT', 'long': 'Full-Time'},
        'whoLostOnAggregated': ''},
       'timeTS': 1637890200000,
       'tv': None},
      {'id': 3714169,
       'leagueId': 875027,
       'time': '25.11.2021 21:00',
       'home': {'id': 8548,
        'score': 2,
        'n

In [32]:
#Filtering down to just shots

shots_data = data['matchFacts']['data']['content']['shotmap']['shots']
shots_data

[{'id': 2358269473,
  'eventType': 'AttemptSaved',
  'teamId': 10260,
  'playerId': 422685,
  'playerName': 'Bruno Fernandes',
  'x': 81.80922502006557,
  'y': 26.858095879232447,
  'min': 86,
  'minAdded': None,
  'isBlocked': True,
  'isOnTarget': True,
  'blockedX': 92.3,
  'blockedY': 30.568749999999998,
  'goalCrossedY': 35.90625,
  'goalCrossedZ': 1.2199999940000001,
  'expectedGoals': 0.0248,
  'expectedGoalsOnTarget': None,
  'shotType': 'RightFoot',
  'situation': 'FromCorner',
  'period': 'SecondHalf',
  'isOwnGoal': False,
  'onGoalShot': {'x': 0.49570105820105825,
   'y': 0.32275132116402117,
   'zoomRatio': 1},
  'firstName': 'Bruno',
  'lastName': 'Fernandes'},
 {'id': 2358269423,
  'eventType': 'AttemptSaved',
  'teamId': 10260,
  'playerId': 696365,
  'playerName': 'Marcus Rashford',
  'x': 95.15789179082141,
  'y': 46.803313167587135,
  'min': 85,
  'minAdded': None,
  'isBlocked': False,
  'isOnTarget': True,
  'blockedX': 103.6724137932,
  'blockedY': 36.51625,
  'go

In [33]:
#Appending the values in list that will be used to create DataFrame

team = []
player = []
x = []
y = []
end_x = []
end_y = []
time = []
result = []
situation = []
xG = []
xGOT = []

for shots in shots_data:
    for details, data in shots.items():
        if details == 'eventType':
            result.append(data)
        if details == 'teamId':
            team.append(data)
        if details == 'playerName':
            player.append(data)
        if details == 'x':
            x.append(data)
        if details == 'y':
            y.append(data)
        if details == 'blockedX':
            if data == None:
                end_x.append(0)
            else: end_x.append(data)
        if details == 'blockedY':
            if data == None:
                end_y.append(0)
            else: end_y.append(data)
        if details == 'min':
            time.append(data)
        if details == 'expectedGoals':
            xG.append(data)
        if details == 'expectedGoalsOnTarget':
            xGOT.append(data)
        if details == 'situation':
            situation.append(data)

In [34]:
#Creating DataFrame

col_names = ['player','team','x','y','endx','endy','result','xg','xgot','time','situation']
df = pd.DataFrame([player,team,x,y,end_x,end_y,result,xG,xGOT,time,situation], index = col_names)
df = df.T
df

,player,team,x,y,endx,endy,result,xg,xgot,time,situation
0,Bruno Fernandes,10260,81.809225,26.858096,92.3,30.56875,AttemptSaved,0.0248,None,86,FromCorner
1,Marcus Rashford,10260,95.157892,46.803313,103.672414,36.51625,AttemptSaved,0.0504,0.1049,85,RegularPlay
2,Cristiano Ronaldo,10260,96.798247,45.682295,0,0,Miss,0.2932,None,84,RegularPlay
3,Etienne Capoue,10205,89.699997,36.134999,0,0,Miss,0.0289,None,83,FromCorner
4,Fred,10260,86.138239,35.525,96.991228,35.3725,AttemptSaved,0.0323,None,81,FromCorner
5,Jadon Sancho,10260,93.5,31.559999,99.307018,33.46625,AttemptSaved,0.3346,0.5596,71,RegularPlay
6,Arnaut Danjuma,10205,92.300003,42.639687,0,0,Miss,0.0633,None,63,RegularPlay
7,Manuel Trigueros,10205,93,29.631667,103.482759,33.16125,AttemptSaved,0.068,0.0403,59,RegularPlay
8,Moi Gomez,10205,86.138239,48.554906,91.5,44.561274,AttemptSaved,0.0332,None,59,RegularPlay
9,Fred,10260,85.064709,24.897813,103.293103,33.00875,AttemptSaved,0.0368,0.0794,53,RegularPlay
